In [1]:
import os
import pandas as pd 
import datetime

os.getcwd()

'/Users/marekerben/Desktop/Prakticka'

In [92]:
#Loading data
quotes_df = pd.read_csv('quotes-binance-btc-2020-11-15.csv.gz')
trades_df = pd.read_csv('trades-binance-btcusdt-2020-11-15.csv.gz')
print(quotes_df.head(), trades_df.head())

          exchange   symbol         timestamp   local_timestamp  ask_amount  \
0  binance-futures  BTCUSDT  1605398403538000  1605398403818508       2.168   
1  binance-futures  BTCUSDT  1605398403837000  1605398403862839       2.168   
2  binance-futures  BTCUSDT  1605398403891000  1605398403904974       2.120   
3  binance-futures  BTCUSDT  1605398403981000  1605398403992118       2.120   
4  binance-futures  BTCUSDT  1605398404056000  1605398404077327       2.120   

   ask_price  bid_price  bid_amount  
0    16068.8   16068.79       3.639  
1    16068.8   16068.79       3.660  
2    16068.8   16068.79       3.771  
3    16068.8   16068.79       3.837  
4    16068.8   16068.79       3.841             exchange   symbol         timestamp   local_timestamp         id  \
0  binance-futures  BTCUSDT  1605398403494000  1605398403504834  267330004   
1  binance-futures  BTCUSDT  1605398403494000  1605398403506164  267330005   
2  binance-futures  BTCUSDT  1605398403494000  1605398403506173

In [93]:
#controlling for NAs
print(quotes_df.isna().sum())

exchange           0
symbol             0
timestamp          0
local_timestamp    0
ask_amount         0
ask_price          0
bid_price          0
bid_amount         0
dtype: int64


In [90]:
#Copying data to new Dataframe
df = quotes_df[['exchange', 'symbol', 'timestamp', 'local_timestamp']].copy()

,exchange,symbol,timestamp,local_timestamp,ask_amount,ask_price,bid_price,bid_amount
0,binance-futures,BTCUSDT,1605398403538000,1605398403818508,2.168,16068.80,16068.79,3.639
1,binance-futures,BTCUSDT,1605398403837000,1605398403862839,2.168,16068.80,16068.79,3.660
2,binance-futures,BTCUSDT,1605398403891000,1605398403904974,2.120,16068.80,16068.79,3.771
3,binance-futures,BTCUSDT,1605398403981000,1605398403992118,2.120,16068.80,16068.79,3.837
4,binance-futures,BTCUSDT,1605398404056000,1605398404077327,2.120,16068.80,16068.79,3.841
...,...,...,...,...,...,...,...,...
947205,binance-futures,BTCUSDT,1605484799686000,1605484799700882,0.001,15952.00,15951.91,0.021
947206,binance-futures,BTCUSDT,1605484799738000,1605484799750045,0.001,15952.00,15951.92,0.052
947207,binance-futures,BTCUSDT,1605484799783000,1605484799795359,0.001,15952.00,15951.93,0.026
947208,binance-futures,BTCUSDT,1605484799826000,1605484799838128,0.001,15952.00,15951.94,0.017


In [97]:
#Computing mid price and delta mid price
df['mid_price'] = (quotes_df['ask_price'] + quotes_df['bid_price']) / 2
df['delta_mid_price'] = df['mid_price'].diff()
df

,exchange,symbol,timestamp,local_timestamp,mid_price,delta_mid_price
0,binance-futures,BTCUSDT,1605398403538000,1605398403818508,16068.795,NaN
1,binance-futures,BTCUSDT,1605398403837000,1605398403862839,16068.795,0.000
2,binance-futures,BTCUSDT,1605398403891000,1605398403904974,16068.795,0.000
3,binance-futures,BTCUSDT,1605398403981000,1605398403992118,16068.795,0.000
4,binance-futures,BTCUSDT,1605398404056000,1605398404077327,16068.795,0.000
...,...,...,...,...,...,...
947205,binance-futures,BTCUSDT,1605484799686000,1605484799700882,15951.955,0.005
947206,binance-futures,BTCUSDT,1605484799738000,1605484799750045,15951.960,0.005
947207,binance-futures,BTCUSDT,1605484799783000,1605484799795359,15951.965,0.005
947208,binance-futures,BTCUSDT,1605484799826000,1605484799838128,15951.970,0.005


In [46]:
#Computing OFI beginning
nrows = quotes_df.shape[0]
e = [None]

for n in range(1, nrows):
    e.append(quotes_df['bid_amount'][n] * (quotes_df['bid_price'][n] >= quotes_df['bid_price'][n-1]) 
             - quotes_df['bid_amount'][n-1] * (quotes_df['bid_price'][n] <= quotes_df['bid_price'][n-1])
             - quotes_df['ask_amount'][n] * (quotes_df['ask_price'][n] <= quotes_df['ask_price'][n-1])
             + quotes_df['ask_amount'][n-1] * (quotes_df['ask_price'][n] >= quotes_df['ask_price'][n-1])
             )

In [98]:
df['e_n'] = e
df

,exchange,symbol,timestamp,local_timestamp,mid_price,delta_mid_price,e_n
0,binance-futures,BTCUSDT,1605398403538000,1605398403818508,16068.795,NaN,NaN
1,binance-futures,BTCUSDT,1605398403837000,1605398403862839,16068.795,0.000,0.021
2,binance-futures,BTCUSDT,1605398403891000,1605398403904974,16068.795,0.000,0.159
3,binance-futures,BTCUSDT,1605398403981000,1605398403992118,16068.795,0.000,0.066
4,binance-futures,BTCUSDT,1605398404056000,1605398404077327,16068.795,0.000,0.004
...,...,...,...,...,...,...,...
947205,binance-futures,BTCUSDT,1605484799686000,1605484799700882,15951.955,0.005,0.021
947206,binance-futures,BTCUSDT,1605484799738000,1605484799750045,15951.960,0.005,0.052
947207,binance-futures,BTCUSDT,1605484799783000,1605484799795359,15951.965,0.005,0.026
947208,binance-futures,BTCUSDT,1605484799826000,1605484799838128,15951.970,0.005,0.017


In [106]:

timestamp = df['timestamp'] / 1000000000
local_timestamp = df['local_timestamp'] / 1000000000
utc_timestamp = []
utc_local_timestamp = []
for i in range(len(timestamp)):
    utc_datetime = datetime.datetime.utcfromtimestamp(timestamp[i]) 
    utc_timestamp.append(utc_datetime.strftime('%Y-%m-%d %H:%M:%S.%f'))
    local_datetime = datetime.datetime.utcfromtimestamp(local_timestamp[i])
    utc_local_timestamp.append(local_datetime.strftime('%Y-%m-%d %H:%M:%S.%f'))

In [107]:
print(len(utc_local_timestamp), len(utc_timestamp))

947210 947210


In [108]:
df['utc_timestamp'] = utc_timestamp
df['utc_local_timestamp'] = utc_local_timestamp
df

,exchange,symbol,timestamp,local_timestamp,mid_price,delta_mid_price,e_n,utc_timestamp,utc_local_timestamp
0,binance-futures,BTCUSDT,1605398403538000,1605398403818508,16068.795,NaN,NaN,1970-01-19 13:56:38.403538,1970-01-19 13:56:38.403819
1,binance-futures,BTCUSDT,1605398403837000,1605398403862839,16068.795,0.000,0.021,1970-01-19 13:56:38.403837,1970-01-19 13:56:38.403863
2,binance-futures,BTCUSDT,1605398403891000,1605398403904974,16068.795,0.000,0.159,1970-01-19 13:56:38.403891,1970-01-19 13:56:38.403905
3,binance-futures,BTCUSDT,1605398403981000,1605398403992118,16068.795,0.000,0.066,1970-01-19 13:56:38.403981,1970-01-19 13:56:38.403992
4,binance-futures,BTCUSDT,1605398404056000,1605398404077327,16068.795,0.000,0.004,1970-01-19 13:56:38.404056,1970-01-19 13:56:38.404077
...,...,...,...,...,...,...,...,...,...
947205,binance-futures,BTCUSDT,1605484799686000,1605484799700882,15951.955,0.005,0.021,1970-01-19 13:58:04.799686,1970-01-19 13:58:04.799701
947206,binance-futures,BTCUSDT,1605484799738000,1605484799750045,15951.960,0.005,0.052,1970-01-19 13:58:04.799738,1970-01-19 13:58:04.799750
947207,binance-futures,BTCUSDT,1605484799783000,1605484799795359,15951.965,0.005,0.026,1970-01-19 13:58:04.799783,1970-01-19 13:58:04.799795
947208,binance-futures,BTCUSDT,1605484799826000,1605484799838128,15951.970,0.005,0.017,1970-01-19 13:58:04.799826,1970-01-19 13:58:04.799838


In [109]:
df.to_csv('Prvni_vypocty.csv.gz', compression = 'gzip')